In [1]:
import pycuda.driver as cuda
import pycuda.autoinit  # Inicializa automaticamente o driver CUDA

# Obtém o dispositivo atual (GPU)
device = cuda.Device(0)

# Informações gerais
print("Nome da GPU:", device.name())
print("Máximo de threads por bloco:", device.get_attribute(cuda.device_attribute.MAX_THREADS_PER_BLOCK))
print("Dimensões máximas dos blocos (x, y, z):", device.get_attribute(cuda.device_attribute.MAX_BLOCK_DIM_X),
      device.get_attribute(cuda.device_attribute.MAX_BLOCK_DIM_Y),
      device.get_attribute(cuda.device_attribute.MAX_BLOCK_DIM_Z))
print("Dimensões máximas da grade (grid) (x, y, z):", device.get_attribute(cuda.device_attribute.MAX_GRID_DIM_X),
      device.get_attribute(cuda.device_attribute.MAX_GRID_DIM_Y),
      device.get_attribute(cuda.device_attribute.MAX_GRID_DIM_Z))

Nome da GPU: NVIDIA GeForce GTX 1650
Máximo de threads por bloco: 1024
Dimensões máximas dos blocos (x, y, z): 1024 1024 64
Dimensões máximas da grade (grid) (x, y, z): 2147483647 65535 65535


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import cmath

from PIL import Image

In [3]:
def crop_center(arr, cropx, cropy):
    y, x = arr.shape[0], arr.shape[1]
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)

    if len(arr.shape) == 2:
       return arr[starty:starty+cropy, startx:startx+cropx]

    return arr[starty:starty+cropy, startx:startx+cropx, :]


# CPU Image Processing

In [4]:
import numpy as np
import cmath

def stockham_fft(signal):
    N = len(signal)

    stages = int(np.log2(N))

    src = signal.copy()
    dst = np.zeros(N, dtype=complex)

    for stage in range(stages):
        m = 2 ** (stage + 1)

        half_m = m // 2

        for idx in range(N // 2):
            src_idx_l = idx
            src_idx_r = idx + N // 2

            group = idx // half_m

            j = idx % half_m

            w = cmath.exp(-2j * cmath.pi * j / m)

            dst[group*m + j] = src[src_idx_l] + w * src[src_idx_r]
            dst[group*m + j + half_m] =  src[src_idx_l] - w * src[src_idx_r]

        src, dst = dst, src

    return src

def stockham_ifft(signal):
    N = len(signal)
    stages = int(np.log2(N))

    src = signal.copy()
    dst = np.zeros(N, dtype=complex)

    for stage in range(stages):
        m = 2 ** (stage + 1)
        half_m = m // 2

        for idx in range(N // 2):
            src_idx_l = idx
            src_idx_r = idx + N // 2

            group = idx // half_m
            j = idx % half_m

            w = cmath.exp(+2j * cmath.pi * j / m)

            dst[group*m + j] = src[src_idx_l] + w * src[src_idx_r]
            dst[group*m + j + half_m] = src[src_idx_l] - w * src[src_idx_r]

        src, dst = dst, src

    return src / N

In [5]:
def fft2_stockham(matrix):
    temp = np.array([stockham_fft(row) for row in matrix])
    result = np.array([stockham_fft(col) for col in temp.T]).T
    return result

def ifft2_stockham(matrix):
    temp = np.array([stockham_ifft(row) for row in matrix])
    result = np.array([stockham_ifft(col) for col in temp.T]).T
    return result

In [6]:
def process_image_cpu(array: np.array):
    fft_img = fft2_stockham(np.array(array, dtype=complex))

    ifft_array = ifft2_stockham(fft_img)

    img = ifft_array.real

    return img

# GPU Image Processing

In [7]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.compiler import SourceModule
import math

mod = SourceModule("""
__device__ float2 complex_mul(float2 a, float2 b) {
    return make_float2(
        a.x * b.x - a.y * b.y,
        a.x * b.y + a.y * b.x
    );
}

                   
__device__ void transpose(float2 *input, int size, int tid) {
    if (tid >= size) return;
    for (int i = tid + 1; i < size; i++) {
        float2 temp = input[tid * size + i];
        input[tid * size + i] = input[i * size + tid];
        input[i * size + tid] = temp;
    }
}

                   
__device__ void vector_flipold(float2 *src, int N) {
    float2 aux;
    for (int i = 0; i < N/4; i++) {
        aux = src[i];
        src[i] = src[N/2-i-1];
        src[N/2-i-1] = aux;
        aux = src[N/2 + i];
        src[N/2 + i] = src[N-i-1];
        src[N-i-1] = aux;
    }
}

                   
__device__ void vector_flip(float2 *src, int N) {
    float2 aux;
    for (int i = 0; i < N/2; i++) {
        aux = src[i];
        src[i] = src[i + N/2];
        src[i + N/2] = aux;
    }
}

                   
__device__ void flip(float2 *src, int N, int tid) {
    vector_flip(&src[tid * N], N);
    __syncthreads();
    transpose(src, N, tid);
    __syncthreads();
    vector_flip(&src[tid * N], N);
    __syncthreads();
    transpose(src, N, tid);
}

                   
__device__ float2 twiddle_pos(int j, int m) {
    float theta = +2.0f * 3.14159265359f * j / m;
    return make_float2(cosf(theta), sinf(theta));
}

                   
__device__ float2 twiddle_neg(int j, int m) {
    float theta = -2.0f * 3.14159265359f * j / m;
    return make_float2(cosf(theta), sinf(theta));
}


__device__ void fft_stockham_serial(float2 *src, float2 *dst, int N) {
    __syncthreads();

    int stages = 0;

    int n = N;

    while (n > 1) {
        n /= 2;
        stages++;
    }

    for (int stage = 0; stage < stages; stage++) {
        int m = 1 << (stage + 1);
        int half_m = m / 2;

        for (int idx = 0; idx < N / 2; idx++) {
            int group = idx / half_m;
            int j = idx % half_m;

            int src_idx_l = idx;
            int src_idx_r = idx + N / 2;

            float2 w = twiddle_neg(j, m);
            float2 a = src[src_idx_l];
            float2 b = complex_mul(w, src[src_idx_r]);

            dst[group * m + j] = make_float2(a.x + b.x, a.y + b.y);
            dst[group * m + j + half_m] = make_float2(a.x - b.x, a.y - b.y);
        }

        for (int i = 0; i < N; i++) {
            src[i] = dst[i];
        }
    }
}

                   
__device__ void ifft_stockham_serial(float2 *src, float2 *dst, int N) {
    __syncthreads();

    int stages = 0;

    int n = N;

    while (n > 1) {
        n /= 2;
        stages++;
    }

    for (int stage = 0; stage < stages; stage++) {
        int m = 1 << (stage + 1);
        int half_m = m / 2;

        for (int idx = 0; idx < N / 2; idx++) {
            int group = idx / half_m;
            int j = idx % half_m;

            int src_idx_l = idx;
            int src_idx_r = idx + N / 2;

            float2 w = twiddle_pos(j, m);
            float2 a = src[src_idx_l];
            float2 b = complex_mul(w, src[src_idx_r]);

            dst[group * m + j] = make_float2(a.x + b.x, a.y + b.y);
            dst[group * m + j + half_m] = make_float2(a.x - b.x, a.y - b.y);
        }

        for (int i = 0; i < N; i++) {
            src[i] = dst[i];
        }
    }

    for (int i = 0; i < N; i++) {
        src[i].x /= N;
        src[i].y /= N;
    }
}


__global__ void fft2_stockham(float2 *src, float2 *dst, int N) {
    int tid = threadIdx.x + blockDim.x * blockIdx.x;

    fft_stockham_serial(&src[tid * N], &dst[tid * N], N);
    __syncthreads();
    transpose(src, N, tid);
    __syncthreads();
    fft_stockham_serial(&src[tid * N], &dst[tid * N], N);
    __syncthreads();
    transpose(src, N, tid);
    __syncthreads();
    flip(src, N, tid);
}

                   
__global__ void ifft2_stockham(float2 *src, float2 *dst, int N) {
    int tid = threadIdx.x + blockDim.x * blockIdx.x;

    flip(src, N, tid);
    __syncthreads();
    ifft_stockham_serial(&src[tid * N], &dst[tid * N], N);
    __syncthreads();
    transpose(src, N, tid);
    __syncthreads();
    ifft_stockham_serial(&src[tid * N], &dst[tid * N], N);
    __syncthreads();
    transpose(src, N, tid);
}
""")

C:\Users\Vitor\AppData\Local\Temp\ipykernel_6424\3244808476.py:7: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod = SourceModule("""


In [8]:
def fft2_gpu (signal):
    N = signal.shape[0]

    src_gpu = cuda.mem_alloc(signal.nbytes)
    dst_gpu = cuda.mem_alloc(signal.nbytes)

    signal = np.ascontiguousarray(signal)

    cuda.memcpy_htod(src_gpu, signal)

    stockham_serial = mod.get_function("fft2_stockham")
    stockham_serial(src_gpu, dst_gpu, np.int32(N), block=(N, 1, 1), grid=(1, 1))

    fft_array = np.empty_like(signal)

    cuda.memcpy_dtoh(fft_array, src_gpu)

    return fft_array


def ifft2_gpu (signal):
    N = signal.shape[0]

    src_gpu = cuda.mem_alloc(signal.nbytes)
    dst_gpu = cuda.mem_alloc(signal.nbytes)

    signal = np.ascontiguousarray(signal)

    cuda.memcpy_htod(src_gpu, signal)

    inv_stockham = mod.get_function("ifft2_stockham")
    inv_stockham(src_gpu, dst_gpu, np.int32(N), block=(N, 1, 1), grid=(1, 1))

    ifft_array = np.empty_like(signal)

    cuda.memcpy_dtoh(ifft_array, src_gpu)

    return ifft_array


In [9]:
def process_image_gpu(array: np.array):
    fft_img = fft2_gpu(np.array(array, dtype=np.complex64))

    ifft_array = ifft2_gpu(fft_img)

    img = ifft_array.real

    return img

# Webcam

In [11]:
import cv2 as cv
    
camera = cv.VideoCapture(0)

camera.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
camera.set(cv.CAP_PROP_FRAME_HEIGHT, 720)

if not camera.isOpened():
    print("Error opening camera.")
    exit()


while True:
    status, frame = camera.read()

    try:
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        array = crop_center(gray_frame, 512, 512)

        #array = process_image_cpu(array)

        array = process_image_gpu(array)

        gray_norm = (array - np.min(array)) / (np.max(array) - np.min(array))
    except Exception as e:
        print(e)
        continue

    if not status or cv.waitKey(1) & 0xff == ord('q'):
        break

    cv.imshow("Camera", gray_norm)

camera.release()

cv.destroyAllWindows()